# HW6 - Transforming vote tallies

This homework give you some insight into the transformations that are underlying all the voting dashboards you've undoubtedly seen all over the news.  Here we are going to use 2020 Presidential election data.

The dataset for this week contains voting outcomes for different batches of votes in the battleground states.  The data consist of the following:
* The state of the votes
* The time the voting results were reported
* The number of votes in the batch (new_votes)
* The number of those new votes that were for Joe Biden (votes_biden)

The key metric of interest is in how each candidate, Biden and Trump, are doing over time in terms of the percentage of votes they're getting with each batch.  This is easier to think of compared to raw numbers for a variety of reasons.  One is that the number of votes varies state-to-state and batch-to-batch, so just thinking about raw numbers doesn't inform very much.  The other is that people are often thinking about the percentage a candidate needs to take the lead or win the state.  If each batch is hitting at or above that percentage, then they have a good chance of winning the state.  

The goal of this homework is to generate the percentages of votes in each state that are going to Trump and Biden **on an hourly reporting basis**. Each line is a batch of votes that were reported at a given time.  Sometimes there are multiple in an hour.  We want to report a coarser resolution which is why we want to group by hours and days.  

Given you have only the variables listed above, you're going to need to do a few things to make this happen:
* Import your data and apply a timestamp
* Make a column for the number of votes received by Trump
* Aggregate your data to get the number of votes for each on a daily and hourly basis
* Create columns of the percentages of votes each have received in the aggregated data

**NOTE 1** - This is by every measure 'small' data, but it needs to be coded in pyspark.

**NOTE 2** - These are all votes that were reported after the initial waves of in-person and early reported mail-in votes.  So if you explore you might see that the total votes in this dataset don't seem to match up with what you would find elsewhere.  That's only because those early votes aren't added in.  These are just batch totals of later reported batches.



**Submission Instruction:**

* Make a copy and replace blank in the title with your name
* Run all the cells
* Download the notebook (.ipynb)
* Submit on Gradescope



In [ ]:
!pip install pyspark

## Importing your data - 3 point

The URL to import your data is

http://131.193.32.85:9000/mybucket/votes_2020_hw5.txt

You need to import the data with the appropriate options applied.  You will need to apply a timestampFormat option as well, but it's best to import the data first, look at the format, and then create your timestamp string.  Note, this is fractional seconds in the timestamp. <a href="https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html" target="_blank">Remember you can see a full list of datetime formatting here</a>

Also, although it's a text file you can import it as a CSV and specify a delimiter as an option.  

Call the imported dataframe 'votes'

In [ ]:
# Make a filepath
...

In [ ]:
# import as votes
...

In [ ]:
# Check data
votes.show()

## Create number of votes for trump - 3 points

Just like it sounds.  Create a column in votes called 'votes_trump' that has the number of votes he received in the batch.

You'll need to import all the functions from pyspark.sql.functions to do this and later steps.

In [ ]:
# make votes_trump

In [ ]:
# Check
...

## Aggregate data - 6 points

Time to group your data. You're going to need to group by day, hour, and then state to get the totals.  I showed you how to group by a single column that's not a datetime in the previous lesson.  Do do more than one you just add them in separated by commas.  The only issue is that you need to extract day and hour from the timestamp.  To do this for the day of the month your grouping variable would be `dayofmonth('timestamp')`.  I'll let you figure out how to do it for hour and day.  Grouping by state should be easy.

For the aggregation you need to calculate the sums that will allow you to get the percentage of votes that went to Biden vs Trump in a given hour.

Call the resulting dataframe votes_hourly

In [ ]:
# make votes_hourly

## Calculating percentage of votes per hour - 6 points

Now go and make your columns of the percentage of votes received each hour by each candidate. Call these 'percent_biden' and 'percent_trump'

## Rounding values - 3 point

Those percent_biden and percent_trump columns have too many digits.  You can import a rounding `round()` function from pyspark using `from pyspark.sql.functions import round`.  Go and import that function and then apply round to those columns so that they have 3 digits after the decimal.  It's fine to overwrite them with the rounded values.

## SQL query - 9 points

Now let's leverage those SQL powers to summarize our data a bit more within Georgia, which was a critical state for 2020 election.  I want you to write two queries.  
1. The first should be just getting all data from Georgia. You can use "Like '%Georgia%' " to match for Georigia state in your query (you may see negative values because the data has not been cleaned).
2. The second should get the daily percent of votes going to Trump for each state. Keep in mind that the percent votes will need to be recalculated (ie averaging the hourly percentages won't work).

Remember you need to register your dataframe as a table to access it!

In [ ]:
# query 1

In [ ]:
# query 2